In [ ]:
# 예제 1: 뉴스 리서치 어시스턴트 - 웹 검색 및 요약 Agent
# Google Colab에서 실행하세요

# ============================
# 1. 필요한 패키지 설치
# ============================
!pip install -q langchain langchain-openai langchain-community gradio duckduckgo-search ddgs
!pip install -q newspaper3k lxml_html_clean
!pip install -q requests



 --> pip install ddgs

In [1]:

# ============================
# 2. 라이브러리 임포트 및 설정
# ============================
import os
import gradio as gr
from typing import List, Dict, Any
from datetime import datetime

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.tools import Tool, StructuredTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.schema import HumanMessage, AIMessage
from newspaper import Article
import json

In [2]:


# ============================
# 3. API 키 설정 (Colab Secrets 또는 직접 입력)
# ============================
# OpenAI API 키 설정 (사용자가 입력해야 함)
# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key
# api_key2=userdata.get('api_key2')
# os.environ["LANGCHAIN_API_KEY"] = api_key2

from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "RAG_Agent_2"                 # 수업용 프로젝트명

In [ ]:
# ============================
# 4. Custom Tools 정의
# ============================

# Tool 1: 웹 검색 도구
search = DuckDuckGoSearchRun()

def search_news(query: str) -> str:
    """최신 뉴스나 정보를 검색합니다."""
    try:
        if not query or query.strip() == "":
            return "검색어를 입력해주세요."

        results = search.run(query)
        if results:
            return results[:2000]  # 토큰 제한을 위해 결과 제한
        else:
            return "검색 결과가 없습니다."
    except Exception as e:
        return f"검색 중 오류 발생: {str(e)}"

search_tool = Tool(
    name="search_news",
    description="Search for recent news, articles, or information about any topic",
    func=search_news
)

# Tool 2: 기사 내용 추출 도구
def extract_article_content(url: str) -> str:
    """URL에서 기사 본문을 추출합니다."""
    try:
        # URL 유효성 검사
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url

        article = Article(url)
        article.download()
        article.parse()

        content = {
            "title": article.title or "제목 없음",
            "authors": article.authors if article.authors else ["저자 정보 없음"],
            "publish_date": str(article.publish_date) if article.publish_date else "날짜 정보 없음",
            "text": article.text[:3000] if article.text else "본문을 추출할 수 없습니다",  # 토큰 제한
            "summary": article.summary if hasattr(article, 'summary') and article.summary else "요약 없음"
        }
        return json.dumps(content, ensure_ascii=False, indent=2)
    except Exception as e:
        # 기본 requests로 폴백
        try:
            import requests
            response = requests.get(url, timeout=10)
            return f"기사 파싱 실패, HTML 일부: {response.text[:500]}..."
        except:
            return f"기사 추출 실패: {str(e)}"

article_tool = Tool(
    name="extract_article",
    description="Extract full article content from a given URL",
    func=extract_article_content
)

# Tool 3: 날짜/시간 확인 도구
def get_current_datetime(query: str = "") -> str:
    """현재 날짜와 시간을 반환합니다."""
    now = datetime.now()
    return f"현재 시각: {now.strftime('%Y년 %m월 %d일 %H시 %M분')}"

datetime_tool = Tool(
    name="get_datetime",
    description="Get current date and time",
    func=get_current_datetime
)


def search_category_new(query: str = "") -> str:
    """query에서 카테고리 정보를 찾아 해당 카테고리의 뉴스 목록을 반환합니다."""
    try:
        if not query or query.strip() == "":
            return "카테고리 검색어를 입력해주세요."

        # 카테고리별 검색 키워드 매핑
        category_map = {
            "정치": ["정치", "politics", "국정감사", "정당", "선거", "정책"],
            "경제": ["경제", "economy", "주식", "환율", "부동산", "금리", "기업"],
            "사회": ["사회", "society", "사건", "사고", "시위", "복지", "교통"],
            "IT": ["IT", "기술", "technology", "스마트폰", "AI", "인공지능", "소프트웨어"],
            "과학": ["과학", "science", "연구", "발견", "실험", "의학"],
            "스포츠": ["스포츠", "sports", "축구", "야구", "올림픽", "경기"],
            "연예": ["연예", "entertainment", "드라마", "K-pop", "영화", "가수"],
            "국제": ["국제", "international", "해외", "외교", "세계", "글로벌"],
            "교육": ["교육", "education", "학교", "입시", "대학", "학생"],
            "문화": ["문화", "culture", "예술", "전시", "공연", "축제"]
        }
        
        # query에서 카테고리 찾기
        detected_category = None
        for category, keywords in category_map.items():
            for keyword in keywords:
                if keyword.lower() in query.lower():
                    detected_category = category
                    break
            if detected_category:
                break
        
        # 카테고리가 감지되지 않으면 일반 검색 수행
        if not detected_category:
            search_query = f"{query} 뉴스 news 최신"
        else:
            # 감지된 카테고리에 맞는 검색 쿼리 생성
            category_keywords = category_map[detected_category]
            search_query = f"{detected_category} 뉴스 {' '.join(category_keywords[:3])} 최신 오늘"
        
        try:
            results = search_with_links.run(query)

            formatted_results = "📰 검색 결과:\n\n"
            for i, result in enumerate(results, 1):
                title = result.get('title', '제목 없음')
                link = result.get('link', '링크 없음')
                snippet = result.get('snippet', '요약 없음')

                formatted_results += f"{i}. **{title}**\n"
                formatted_results += f"   🔗 링크: {link}\n"
                formatted_results += f"   📝 요약: {snippet}\n\n"

            return formatted_results[:2000]  # 토큰 제한
        except Exception as e:
            return f"검색 중 오류 발생: {str(e)}"

        
    except Exception as e:
        return f"카테고리 뉴스 검색 중 오류 발생: {str(e)}"
    
category_news_tool = Tool(
    name="search_category_news",
    description="Searches for news in the requested category and returns a list of articles",
    func=search_category_new
)

from langchain_community.tools import DuckDuckGoSearchResults

# 링크와 제목을 포함한 검색 도구
search_with_links = DuckDuckGoSearchResults(
    num_results=5,  # 결과 개수
    output_format="list"  # 리스트 형태로 출력
)

      
# ============================
# 5. Agent 설정
# ============================

# LLM 초기화 (streaming을 False로 설정하여 안정성 향상)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    streaming=False  # Gradio와의 호환성을 위해 False로 설정
)

# Tools 리스트
tools = [search_tool, article_tool, datetime_tool, category_news_tool]

# Prompt Template
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 뉴스 리서치 전문가입니다.
    사용자의 질문에 대해 최신 정보를 검색하고, 관련 기사를 분석하여
    정확하고 유용한 정보를 제공합니다.

    작업 순서:
    1. 먼저 현재 시간을 확인합니다
    2. 관련 정보를 검색합니다
    3. 필요시 URL에서 상세 내용을 추출합니다
    4. 수집한 정보를 종합하여 답변합니다

    항상 한국어로 답변하세요."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Agent 생성
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

# ============================
# 6. Gradio 인터페이스
# ============================

def process_query(message, history):
    """사용자 쿼리를 처리하고 응답을 생성합니다."""
    try:
        # 대화 기록 변환
        chat_history = []
        if history:  # history가 None이 아닌 경우만 처리
            for h in history:
                if isinstance(h, (list, tuple)) and len(h) >= 2:
                    if h[0]:  # user message
                        chat_history.append(HumanMessage(content=h[0]))
                    if h[1]:  # assistant message
                        chat_history.append(AIMessage(content=h[1]))

        # Agent 실행
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history
        })

        return response["output"]

    except Exception as e:
        import traceback
        error_detail = traceback.format_exc()
        print(f"Error details: {error_detail}")  # 디버깅용
        return f"오류가 발생했습니다: {str(e)}\n\n디버그 정보를 콘솔에서 확인하세요."

# Gradio UI 생성
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🔍 뉴스 리서치 어시스턴트

    최신 뉴스와 정보를 검색하고 분석해드립니다.

    **사용 가능한 기능:**
    - 실시간 웹 검색
    - 기사 내용 추출 및 요약
    - 현재 시간 확인

    **예시 질문:**
    - "최근 AI 관련 주요 뉴스를 알려줘"
    - "OpenAI의 최신 발표 내용은?"
    - "오늘 주식시장 동향을 요약해줘"
    """)

    chatbot = gr.Chatbot(
        height=400,
        bubble_full_width=False,
        avatar_images=(None, "🤖")
    )

    msg = gr.Textbox(
        label="질문을 입력하세요",
        placeholder="예: 최근 테슬라 관련 뉴스를 검색해줘",
        lines=2
    )

    with gr.Row():
        submit = gr.Button("전송", variant="primary")
        clear = gr.Button("대화 초기화")

    # 예시 질문들
    gr.Examples(
        examples=[
            "오늘의 주요 IT 뉴스를 요약해줘",
            "최근 ChatGPT 관련 소식을 알려줘",
            "한국 경제 현황에 대해 검색해줘",
            "현재 시간과 날씨 정보를 알려줘"
        ],
        inputs=msg
    )

    # 이벤트 핸들러
    def respond(message, chat_history):
        bot_message = process_query(message, chat_history)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit.click(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

# ============================
# 7. 실행
# ============================
if __name__ == "__main__":
    print("뉴스 리서치 어시스턴트를 시작합니다...")
    demo.launch(share=True, debug=True)

/var/folders/xn/w4kqb1715fz7h49tp_h7hbmr0000gn/T/ipykernel_60860/1509524029.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/var/folders/xn/w4kqb1715fz7h49tp_h7hbmr0000gn/T/ipykernel_60860/1509524029.py:282: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


뉴스 리서치 어시스턴트를 시작합니다...
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2ba9216674d9338ff6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...

Invoking: `get_datetime` with `Asia/Seoul`


현재 시각: 2025년 09월 02일 15시 37분
Invoking: `search_category_news` with `IT`


📰 검색 결과:

1. **常常聽到 IT ，但您真的了解 IT 資訊科技是什麼嗎？ - iT 邦幫忙 ...**
   🔗 링크: https://ithelp.ithome.com.tw/articles/10311226
   📝 요약: Feb 18, 2023 · 這篇文章將和大家介紹我們常常聽到的一個英文縮寫「IT」，我們總是會在公司、職業的分工或專長，聽到 IT 這個詞彙。著名平台 iTHome 的 …

2. **信息技术（IT） - 知乎**
   🔗 링크: https://www.zhihu.com/topic/19556498/intro
   📝 요약: Apr 24, 2020 · 信息技术（英语：Information Technology，缩写：IT）也称信息和通信技术（Information and Communications …

3. **2025 iThome 鐵人賽 - iT 邦幫忙**
   🔗 링크: https://ithelp.ithome.com.tw/2025ironman/
   📝 요약: 2025 iThome 鐵人賽將於 8 月 1 日開賽，今年除了主題競賽、團體競賽及自我挑戰等3大類型，還全新推出「iThome 鐵人館」邀請大家一起來參加，無論是挑戰 …

4. **用MCP啟動你的AI超能力！架構、應用與資安一次看懂 - iT 邦幫忙**
   🔗 링크: https://ithelp.ithome.com.tw/articles/10373013
   📝 요약: Aug 12, 2025 · 結語 MCP 為 AI 打造一條安全、統一且可擴充的「資料高速公路」： 對開發者：一次學會標準，即可串接海量工具。 對企業：降低整合成本、減 …

5. **iT 邦幫忙::一起幫忙解決難題，拯救 IT 人的一天**
   🔗 링크: https://ithelp.ithome.com.tw

/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Invoking: `search_news` with `IT news`




...

/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:




> Entering new AgentExecutor chain...

Invoking: `search_news` with `ChatGPT`


Aug 24, 2025 · 本文提供 ChatGPT 中文版 使用指南，推荐 国内直连 的 ChatGPT 镜像网站， 支持GPT-4，无需翻墙。 本项目为用户提供全面的 ChatGPT 中文版 使用指南，同时整理了国内可用的 ChatGPT镜像网站 和 官网使用教程，帮助您快速上手 ChatGPT，无论是个人使用还是专业需求。 About 🔮 ChatGPT Desktop Application (Mac, Windows and Linux) desktop-app windows macos linux rust application app ai webview openai gpt notes-app tauri gpt-3 chatgpt Readme Aug 26, 2025 · ChatGPT 镜像网站 （Mirror Site）是一个为满足特定用户使用需求而搭建的访问站点，通常将原始服务同步到国内服务器，优化访问速度，提升网络连接性能。 在国内，ChatGPT 的官方站点可能因网络限制常常无法顺畅使用，而镜像站提供了一种无需翻墙、直接登录的解决方案，并增强了 GPT 的适用性。 Aug 7, 2025 · 本文教你如何在国内便捷使用 ChatGPT 中文版 的方法，并推荐多个 无需翻墙的 ChatGPT 镜像网站。 更新日期: 2025/08/07 在这里，您将找到详尽的 ChatGPT 中文版使用指南，帮您在学习、工作和日常生活中充分利用 ChatGPT 的强大功能。 The ultimate ChatGPT Jailbreak Tool with stunning themes, categorized prompts, and a user-friendly interface. - Batlez/ChatGPT-Jailbreak-Pro

/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


최근 ChatGPT와 관련된 소식은 다음과 같습니다:

1. **ChatGPT 사용 가이드**: 최근에 ChatGPT의 중국어 버전 사용을 위한 종합적인 가이드가 제공되었습니다. 이 가이드는 사용자들이 ChatGPT를 보다 쉽게 사용할 수 있도록 도와주며, 특히 중국 내에서 접근할 수 있는 여러 대체 사이트와 사용 방법을 안내하고 있습니다.

2. **접속 속도 개선**: ChatGPT의 공식 사이트가 중국 내에서 종종 느린 속도를 보이기 때문에, 여러 '미러 사이트'가 개발되었습니다. 이 사이트들은 원래의 ChatGPT 서비스를 중국 서버에 동기화하여 접속 속도를 개선하고, 사용자가 VPN 없이도 원활하게 사용할 수 있도록 돕고 있습니다.

이 외에도 ChatGPT의 다양한 애플리케이션과 사용자 경험을 개선하기 위한 도구들이 지속적으로 개발되고 있습니다.

> Finished chain.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2ba9216674d9338ff6.gradio.live


/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/gradio/tunneling.py:121: ResourceWarning: unclosed file <_io.BufferedReader name=79>
  self.proc = None
/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/gradio/tunneling.py:121: ResourceWarning: unclosed file <_io.BufferedReader name=103>
  self.proc = None


In [13]:
demo.close()

Closing server running on port: 7860
